<a href="https://colab.research.google.com/github/marceloflores-soa/EA3-Colab/blob/main/SOA_Ejercicio_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Descripción Ejecicio 2
El ejercicio posee dos versiones del algoritmo para la multiplicación de matrices, la primera de ellas deberá ejecutarse en el entorno CPU con el lenguaje Python y la segunda en el entorno GPU utilizando el lenguaje Cuda junto con la librería pycuda.

La finalidad del ejercicio es mostrar la ejecución de un algoritmo de multiplicación de matrices para CPU y GPU.

Para poder realizar esta multiplicación se realiza una validación para que esta pueda realizarse. Si tenemos una matriz A de m filas y **n columnas**, sólo la podremos multiplicar por una matriz B que tenga **n filas** y p columnas (mismo número de columnas de A, que de filas de B):

multiplicacion-2.png




## Instalación de PyCuda

In [ ]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 15.7MB/s 
     |████████████████████████████████| 71kB 11.6MB/s 
     |████████████████████████████████| 81kB 12.3MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=620900 sha256=b30f6edf5eacdb9dc7878532e8a30394da789f0c4f933287719eee19a5aaf393
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=3b0155dd2024c4bf77060196cdcac73ef42f21a715f8e6f945ddc409e2ae22ab
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


## Desarrollo CPU

In [ ]:
# --------------------------------------------
#@title Ingrese filas y columas de cada matriz: { vertical-output: true }

matriz_A_filas =   2#@param {type: "number"}
matriz_B_columnas = 2#@param {type: "number"}
columnas_A_filas_B = 2#@param {type: "number"}

if (columnas_A_filas_B <= 0 or matriz_A_filas < 0 or matriz_B_columnas < 0 ):
  raise Exception("Por favor, no ingrese números negativos.") 

# ---Importamos las librerías de CUDA
import numpy as np
from datetime import datetime

# ---Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

# ---MAIN
tiempo_total = datetime.now()
# Generamos matrices con valores entre 0 y 100
matriz_A = np.random.rand(matriz_A_filas, columnas_A_filas_B).astype(np.float32)
matriz_B = np.random.rand(columnas_A_filas_B, matriz_B_columnas).astype(np.float32)
resultado = np.zeros((matriz_A_filas,matriz_B_columnas), dtype=float)

for i in range(len(matriz_A)):
   for j in range(len(matriz_B[0])):
       for k in range(len(matriz_B)):
           resultado[i][j] += matriz_A[i][k] * matriz_B[k][j]

tiempo_total = datetime.now() - tiempo_total

print( "Resultados" )
print( "------------------------------------")
print("Matriz A: ")
print(np.array2string(matriz_A, precision=3, separator=',', suppress_small=True))
print()
print("Matriz B: ")
print(np.array2string(matriz_B, precision=3, separator=',', suppress_small=True))
print()
print("Matriz Resultante: ")
print(np.array2string(resultado, precision=3, separator=',', suppress_small=True))
print()
print( "Tiempos")
print( "------------------------------------")
# Informo tiempos
print("Tiempo CPU: ", tiempo_en_ms( tiempo_total ), "[ms]" )


Resultados
------------------------------------
Matriz A: 
[[0.705,0.399],
 [0.15 ,0.514]]

Matriz B: 
[[0.479,0.227],
 [0.768,0.581]]

Matriz Resultante: 
[[0.644,0.392],
 [0.466,0.333]]

Tiempos
------------------------------------
Tiempo CPU:  0.224 [ms]


# Dessarrollo GPU

* Para poder ejecutar el siguiente desarrollo, por favor, cambie el **tipo de entorno de ejecución** a GPU. Ingresando en "Herramientas" -> "Tipo de entorno de ejecución".

In [ ]:
# --------------------------------------------
#@title Ingrese filas y columas de cada matriz: { vertical-output: true }

matriz_A_filas =   2#@param {type: "number"}
matriz_B_columnas = 2#@param {type: "number"}
columnas_A_filas_B = 2#@param {type: "number"}

if (columnas_A_filas_B <= 0 or matriz_A_filas < 0 or matriz_B_columnas < 0 ):
  raise Exception("Por favor, no ingrese números negativos. El atributo Filas_B_Columnas_A no puede ser igual a 0") 

# ---Importamos librerias PyCuda
import numpy as np
import time
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
from datetime import datetime

# ----Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

tiempo_total = datetime.now()

MATRIX_SIZE = matriz_A_filas * matriz_B_columnas
BLOCK_SIZE = 32

#--- MAIN

# CPU - Defino la memoria de los vectores en cpu.
matriz_A = np.random.randn(matriz_A_filas, columnas_A_filas_B)*10
matriz_A = matriz_A.astype(np.float32)

matriz_B = np.random.randn(columnas_A_filas_B, matriz_B_columnas)*10
matriz_B = matriz_B.astype(np.float32)

resultado = np.zeros((matriz_A_filas,matriz_B_columnas), dtype=float)
resultado = resultado.astype(np.float32)

# CPU - reservo la memoria GPU
a_gpu = cuda.mem_alloc(matriz_A.nbytes)
b_gpu = cuda.mem_alloc(matriz_B.nbytes)
c_gpu = cuda.mem_alloc(resultado.nbytes)

# GPU - Copio la memoria al GPU
cuda.memcpy_htod(a_gpu, matriz_A)
cuda.memcpy_htod(b_gpu, matriz_B)

module = SourceModule("""

__global__ void MatrixMulKernel(int n, float *A, float *B, float *C){

  // Índice de hilos
  int tx = threadIdx.x;
  int ty = threadIdx.y;
  
  // Índice de bloques
  int bx = blockIdx.x;
  int by = blockIdx.y;

  int row = by*blockDim.y + ty;
  int col = bx*blockDim.x + tx;

  if(row < n && col < n){
    float val = 0.0;
    for(int i=0; i<n; ++i){

      //cada hilo carga un elemento de cada matriz
      val += A[row*n + i]*B[n*i + col];
      __syncthreads();
    }
    C[row*n + col] = val;
  }
}

""") 

# Definición funcion Kernel
kernel = module.get_function("MatrixMulKernel")

if MATRIX_SIZE%BLOCK_SIZE != 0:
    grid=(MATRIX_SIZE//BLOCK_SIZE+1,MATRIX_SIZE//BLOCK_SIZE+1,1)
else:
    grid=(MATRIX_SIZE//BLOCK_SIZE,MATRIX_SIZE//BLOCK_SIZE,1)

tiempo_gpu = datetime.now()
# Ejecución entorno GPU
kernel(np.int32(MATRIX_SIZE), a_gpu, b_gpu, c_gpu, block=( BLOCK_SIZE, BLOCK_SIZE, 1 ), grid=grid);
tiempo_gpu = datetime.now() - tiempo_gpu

# GPU - Copio el resultado desde la memoria GPU.
cuda.memcpy_dtoh(resultado, c_gpu)

# Informo resultados
print( "Resultados" )
print( "------------------------------------")
print("Matriz A: ")
print(np.array2string(matriz_A, precision=3, separator=',', suppress_small=True))
print()
print("Matriz B: ")
print(np.array2string(matriz_B, precision=3, separator=',', suppress_small=True))
print()
print("Matriz Resultante: ")
print(np.array2string(resultado, precision=3, separator=',', suppress_small=True))
print()

tiempo_total = datetime.now() - tiempo_total

# Informo tiempos
print( "Tiempos" )
print( "------------------------------------")
print("Tiempo CPU: ", tiempo_en_ms( tiempo_total ), "[ms]" )
print("Tiempo GPU: ", tiempo_en_ms( tiempo_gpu   ), "[ms]") 
print( "------------------------------------")


Resultados
------------------------------------
Matriz A: 
[[ 6.469, 2.886],
 [-3.035, 0.164]]

Matriz B: 
[[  5.749,  0.897],
 [-16.77 , 25.109]]

Matriz Resultante: 
[[  37.188,   5.805],
 [-108.476, 162.419]]

Tiempos
------------------------------------
Tiempo CPU:  927.974 [ms]
Tiempo GPU:  1.795 [ms]
------------------------------------


## Tablas de pasos

### *CPU

 Procesador | Función | Detalle
------------|---------|----------
CPU      |  @param                | Lectura del tamaño de vectores desde Colab.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  raise Exception()     | Lanza una exception.
CPU      |  np.random.randn(X,Y).astype(np.float32) | Inicializa los vectores **matriz_A y matriz_B** con cantidad_N de números random entre el 0 y el 1.
CPU      |  tiempo_en_ms      | Transforma el tiempo en milisegundos.
CPU      |  array2string      | Transforma el array en string con 3 decimales.
CPU      |  print()               | Informo los resultados.

### *GPU

 Procesador | Funciòn | Detalle
------------|---------|----------
CPU      |  @param                | Lectura del tamaño de vectores desde Colab.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  raise Exception()     | Lanza una exception.
CPU      |  np.random.randn(X,Y) | Inicializa los vectores matriz_A y matriz_B con números random entre el 0 y el 1.
CPU      |  np.astype(float32)            | Defino los valores dentro de los vectores como float32.
CPU      |  np.zeros | Genera una matriz de ceros.
**GPU**  |  pycuda.driver.in()      | Indica que el array debe copiarse en el dispositivo de cómputo antes de invocar el kernel.
**GPU**  |  pycuda.driver.out()    | Indica que el array debe copiarse del dispositivo de cómputo después de invocar el kernel.
CPU      |  SourceModule()        | Define el código del kernel. 
CPU      |  module.get_function() | Genera la función del kernel GPU.
**GPU**  |  kernel()              | Ejecuta el kernel en GPU.
CPU      |  tiempo_en_ms()               | Transforma el tiempo en milisegundos.
CPU      |  array2string      | Transforma el array en string con 3 decimales.
CPU      |  print()               | Informo los resultados.

## Conclución

Al ejecutar el algoritmo de multiplicación de matrices en **[1]** CPU  y **[2]** GPU, ingresando un valores altos para las matrices.
**[3]** La implementación realizada con el procesador GPU da una clara ventaja utilizando su paralelismo y logra tiempos mucho más cortos para realizar el producto de matrices en los test realizados con valores grandes y ejecución de hilos 1024 (bloques de tamaño 32 x 32); **[4]** ya que cada hilo será el encargado de calcular el valor resultante en la nueva matriz, esto quiere decir que se generarán tantos hilos como posiciones posea la matriz resultante (filas de la matriz A y columnas de la matriz B).

**[5]** Este ejecicio demuestra el poder que tiene GPU en el paralelismo, lo que puede aplicarse en el ambito de Data Science y Data Mining. Por ejemplo utilizando la librería Rapids. 
Por mi parte, desconocia en profundidad los beneficios del procesamiento GPU, es muy interesante aprender sobre este tema ya que me estoy oriendando hacia la rama de Data Science para el futuro. Estuve implementando en Colab algunos algoritmos en base a la libreria Pandas, utilizando algoritmos de Skylearn de Arboles de decision, KNN y Regresión lineal. Profundizare en el futuro la ejecución de algun algoritmo para GPU.



## Referencias Leídas

La investigación para la resolución de los problemas la obtuve del material de clase, documentación de pycuda y diversas webs en la que se trataban ejemplos y comparaciones entre ejecuciones cpu y gpu.

[1] https://www.cartagena99.com/recursos/alumnos/apuntes/introduccion%20matrices.pdf

[2] https://www.tutorialspoint.com/cuda/cuda_matrix_multiplication.htm

[3] http://www.ijcee.org/vol9/949-E1621.pdf

[4] https://shephexd.github.io/development/2017/02/19/pycuda.html

[5] https://developer.nvidia.com/rapids
